# Old method vs new method

This notebook compares the old equations that we used in the Kalman filter vs. the ones that we use now.

---

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
h = 1e-12
measurement_noise = 1e-11
seed = 1237

In [3]:
import logging 
logger = logging.getLogger().setLevel(logging.ERROR)

In [4]:
#Old method 
from tmp_code.system_parameters import SystemParameters as OldSystemParameters
from tmp_code.model import LinearModel as OldLinearModel
from tmp_code.kalman_filter import KalmanFilter as OldKalmanFilter
from tmp_code.pulsars import Pulsars as OldPulsars
from tmp_code.synthetic_data import SyntheticData as OldSyntheticData
from tmp_code.priors import bilby_priors_dict as Oldbilby_priors_dict

P    = OldSystemParameters(h=h,σp=None,σm=measurement_noise,seed=seed,Npsr=2)       #define the system parameters as a class

PTA  = OldPulsars(P)               # setup the PTA
old_data = OldSyntheticData(PTA,P)     # generate some synthetic data


#Define the model 
model = OldLinearModel(P)

#Initialise the Kalman filter
KF = OldKalmanFilter(model,old_data.f_measured,PTA)

#Sample from a dict prior 
init_p,parameter_priors = Oldbilby_priors_dict(PTA,P)
parameters_sample = parameter_priors.sample(1)
old_ll,xx,yy = KF.likelihood(parameters_sample)



INFO:root:Welcome to the Kalman Filter Nested Sampler for PTA GW systems
INFO:root:Random seed is 1237
INFO:root:You are assigning the σp terms randomly
INFO:root:You are including the PSR terms in your synthetic data generation
INFO:root:Setting the bilby priors dict
INFO:root:Using the GW priors for the measurement model
INFO:root:Sigma p true value is sigma_p0 9.769327690311607e-20
INFO:root:Sigma p true value is sigma_p1 3.8639377766942156e-20


In [5]:
#New method 

import sys
import json 
import pandas as pd 
try:
    sys.path.remove("../py_src") # Hacky way to add higher directory to python modules path. 
except:
    pass
sys.path.append("../py_src") # Means that I dont have to make src/ a proper python package





from system_parameters import SystemParameters as NewSystemParameters
from model import LinearModel as NewLinearModel
from kalman_filter import KalmanFilter as NewKalmanFilter
from pulsars import Pulsars as NewPulsars
from synthetic_data import SyntheticData as NewSyntheticData
from priors import bilby_priors_dict as Newbilby_priors_dict

P    = NewSystemParameters(h=h,σp=None,σm=measurement_noise,seed=seed,Npsr=2)       #define the system parameters as a class

PTA  = NewPulsars(P)               # setup the PTA
new_data = NewSyntheticData(PTA,P)     # generate some synthetic data


#Define the model 
model = NewLinearModel(P)

#Initialise the Kalman filter
KF = NewKalmanFilter(model,new_data.f_measured,PTA)

#Sample from a dict prior 
new_ll = KF.likelihood(parameters_sample)



INFO:root:Welcome to the Kalman Filter Nested Sampler for PTA GW systems
INFO:root:Random seed is 1237
INFO:root:You are assigning the σp terms randomly
INFO:root:You are including the PSR terms in your synthetic data generation
INFO:root:You are using the Pulsar terms measurement model


In [6]:
#Is the data the same?
import numpy as np 
np.allclose(new_data.f_measured, old_data.f_measured)

True

In [13]:
#Are the likelihoods the same?
np.isclose(old_ll,new_ll)

True